In [1]:
from datasets.karel import parser_for_synthesis, mutation, karel_runtime
from datasets import dataset, executor

import collections
import copy
import json
import cPickle as pickle
import pprint
import operator

import numpy as np

In [31]:
def visualize_trace(trace):
    kr = karel_runtime.KarelRuntime()
    event_idx = 0
    for i, grid in enumerate(trace.grids):
        while event_idx < len(trace.events) and trace.events[event_idx].timestep == i:
            print trace.events[event_idx]
            event_idx += 1
        field = np.zeros((15, 18, 18), dtype=np.bool)
        field.ravel()[grid] = True
        kr.init_from_array(field)
        kr.draw()
    while event_idx < len(trace.events):
        print trace.events[event_idx]
        event_idx += 1

def visualize_grid(grid):
    kr = karel_runtime.KarelRuntime()
    field = np.zeros((15, 18, 18), dtype=np.bool)
    field.ravel()[grid] = True
    kr.init_from_array(field)
    kr.draw()

In [2]:
dses = [dataset.KarelTorchDataset(
    'data/karel/train.pkl', mutation.KarelExampleMutator([0.0] * i + [1.0], rng_fixed=True, add_trace=True)) for i in range(3)]
the_executor = executor.KarelExecutor()

In [3]:
all_results = []

for i in range(10000):
    # First get the original trace lengths
    for dist in range(4):
        example = dses[max(dist - 1, 0)][i]
        if dist > 0:
            mutated_tests = example.ref_example.input_tests
            for test_idx, test in enumerate(mutated_tests):
                trace_length = len(test['trace'].grids) + len(test['trace'].events)
                last_event = test['trace'].events[-1]
                all_results.append({
                    'idx': i, 'test_idx': test_idx, 'dist': dist,
                    'trace_length': trace_length, 'last_event': last_event,
                    'example': example})
            continue
            
        for test_idx in range(5):
            result = the_executor.execute(
                example.code_sequence, None, example.input_tests[test_idx]['input'], record_trace=True, strict=True)
        
            trace_length = len(result.trace.grids) + len(result.trace.events)
            last_event = result.trace.events[-1]
        
            all_results.append(
                {'idx': i, 'test_idx': test_idx, 'dist': 0, 'trace_length': trace_length, 'last_event': last_event})        
        
    if i % 1000 == 0: print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


## Length distribution for unmodified code

In [65]:
# Run 1
for dist in range(4):
    print np.percentile([r['trace_length'] for r in all_results if r['dist'] == dist], [50, 90, 99, 99.9, 99.99, 100])

[  13.       47.      132.      196.001   251.0004  255.    ]
[   10.       40.      129.      294.      621.002  1206.   ]
[    9.        36.       122.       301.002    746.0054  1132.    ]
[    8.        34.       114.       306.005    945.0012  1278.    ]


In [75]:
# Run 2
for dist in range(4):
    print np.percentile([r['trace_length'] for r in all_results if r['dist'] == dist], [50, 90, 99, 99.9, 99.99, 100])

[  13.       47.      132.      196.001   251.0004  255.    ]
[   10.        40.       134.       304.       886.0045  1406.    ]
[    9.        36.       126.       304.      1072.0057  1210.    ]
[    8.        34.       115.       358.002    754.0053  1278.    ]


In [4]:
# Run 3, with illegal actions gone + shorter cap
for dist in range(4):
    print np.percentile([r['trace_length'] for r in all_results if r['dist'] == dist], [50, 90, 99, 99.9, 99.99, 100])

[  13.       47.      132.      196.001   251.0004  255.    ]
[  10.       39.      114.      217.002   685.0005  821.    ]
[   9.       34.       98.      216.003   407.0009  693.    ]
[   8.      31.      97.     230.     363.001  727.   ]


## What long traces end in

In [70]:
for dist in range(4):
    results = [r for r in all_results if r['dist'] == dist]
    results.sort(key=lambda r: r['trace_length'], reverse=True)
    for r in results[:10]:
        print dist, r['trace_length'], r['idx'], r['test_idx'], r['last_event']

0 255 8913 0 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 1 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 2 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 3 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 4 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 251 9823 0 KarelEvent(timestep=74, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 1 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 2 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 3 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_spa

In [76]:
for dist in range(4):
    results = [r for r in all_results if r['dist'] == dist]
    results.sort(key=lambda r: r['trace_length'], reverse=True)
    for r in results[:10]:
        print dist, r['trace_length'], r['idx'], r['test_idx'], r['last_event']

0 255 8913 0 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 1 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 2 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 3 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 4 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 251 9823 0 KarelEvent(timestep=74, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 1 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 2 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 3 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_spa

In [5]:
for dist in range(4):
    results = [r for r in all_results if r['dist'] == dist]
    results.sort(key=lambda r: r['trace_length'], reverse=True)
    for r in results[:10]:
        print dist, r['trace_length'], r['idx'], r['test_idx'], r['last_event']

0 255 8913 0 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 1 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 2 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 3 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 255 8913 4 KarelEvent(timestep=72, type='repeat', span=(28, 32), cond_span=(29, 29), cond_value=0, success=True)
0 251 9823 0 KarelEvent(timestep=74, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 1 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 2 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_span=None, cond_value=None, success=True)
0 245 9823 3 KarelEvent(timestep=71, type='turnLeft', span=(30, 30), cond_spa

In [6]:
results = [r for r in all_results if r['dist'] == 1]
results.sort(key=lambda r: r['trace_length'], reverse=True)
r = results[0]
print r['trace_length'], r['idx'], r['test_idx'], r['last_event']

821 5936 0 KarelEvent(timestep=321, type='while', span=(12, 41), cond_span=(14, 14), cond_value=True, success=False)


In [7]:
print ' '.join(r['example'].ref_example.code_sequence)

DEF run m( REPEAT R=8 r( REPEAT R=5 r( putMarker putMarker turnLeft WHILE c( markersPresent c) w( turnLeft turnLeft IF c( leftIsClear c) i( turnLeft move i) IFELSE c( not c( frontIsClear c) c) i( turnLeft i) ELSE e( turnLeft e) w) putMarker turnRight r) r) turnLeft move move putMarker putMarker m)


In [11]:
visualize_trace(r['example'].ref_example.input_tests[0]['trace'])

█████████
█       █
█       █
█  1    █
█       █
█ 1 3   █
█ 1   1 █
█     3 █
█  ↑ 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=1, type='repeat', span=(3, 45), cond_span=(4, 4), cond_value=8, success=True)
KarelEvent(timestep=1, type='repeat', span=(6, 44), cond_span=(7, 7), cond_value=5, success=True)
KarelEvent(timestep=1, type='putMarker', span=(9, 9), cond_span=None, cond_value=None, success=True)
█████████
█       █
█       █
█  1    █
█       █
█ 1 3   █
█ 1   1 █
█     3 █
█  1̅ 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=2, type='putMarker', span=(10, 10), cond_span=None, cond_value=None, success=True)
█████████
█       █
█       █
█  1    █
█       █
█ 1 3   █
█ 1   1 █
█     3 █
█  2̅ 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=3, type='turnLeft', span=(11, 11), cond_span=None, cond_value=None, success=True)
█████████
█       █
█       █
█  1    █
█       █
█ 1 3   █
█ 1   1 █
█     

█ 133  3█
█ 13  13█
█  3  33̲█
█  2 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=174, type='turnLeft', span=(11, 11), cond_span=None, cond_value=None, success=True)
█████████
█  33333█
█  3   3█
█  1   3█
█  3   3█
█ 133  3█
█ 13  13█
█  3  33͕█
█  2 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=175, type='while', span=(12, 41), cond_span=(14, 14), cond_value=True, success=True)
KarelEvent(timestep=175, type='turnLeft', span=(17, 17), cond_span=None, cond_value=None, success=True)
█████████
█  33333█
█  3   3█
█  1   3█
█  3   3█
█ 133  3█
█ 13  13█
█  3  33̅█
█  2 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=176, type='turnLeft', span=(18, 18), cond_span=None, cond_value=None, success=True)
█████████
█  33333█
█  3   3█
█  1   3█
█  3   3█
█ 133  3█
█ 13  13█
█  3  33͔█
█  2 4  █
█       █
█       █
█ 11    █
█       █
█████████
KarelEvent(timestep=177, type='if', span=(19, 26), cond_span=(21, 21)

KarelEvent(timestep=302, type='turnLeft', span=(18, 18), cond_span=None, cond_value=None, success=True)
█████████
█  33333█
█  3   3█
█  1   3█
█  3   3█
█ 133  3█
█ 13  13█
█  3  33█
█  2 4 3█
█      3█
█      3█
█3͕11   3█
█3333333█
█████████
KarelEvent(timestep=303, type='if', span=(19, 26), cond_span=(21, 21), cond_value=True, success=True)
KarelEvent(timestep=303, type='turnLeft', span=(24, 24), cond_span=None, cond_value=None, success=True)
█████████
█  33333█
█  3   3█
█  1   3█
█  3   3█
█ 133  3█
█ 13  13█
█  3  33█
█  2 4 3█
█      3█
█      3█
█3̅11   3█
█3333333█
█████████
KarelEvent(timestep=304, type='move', span=(25, 25), cond_span=None, cond_value=None, success=True)
█████████
█  33333█
█  3   3█
█  1   3█
█  3   3█
█ 133  3█
█ 13  13█
█  3  33█
█  2 4 3█
█      3█
█↑     3█
█311   3█
█3333333█
█████████
KarelEvent(timestep=305, type='ifElse', span=(27, 40), cond_span=(29, 32), cond_value=False, success=True)
KarelEvent(timestep=305, type='turnLeft', span=(39, 39), cond

In [ ]:
results = [r for r in all_results if r['dist'] == 1]
results.sort(key=lambda r: r['trace_length'], reverse=True)
r = results[0]
print r['trace_length'], r['idx'], r['test_idx'], r['last_event']

In [73]:
dses[1][8580].ref_example.input_tests[0]['trace'].events[-1]

KarelEvent(timestep=39, type='turnLeft', span=(64, 64), cond_span=None, cond_value=None, success=True)

In [53]:
results = [r for r in all_results if r['dist'] == 3]
print results[25]
ex = results[25]['example']
print ' '.join(ex.ref_example.code_sequence)
print ' '.join(ex.code_sequence)

for i in range(5):
    print 'Input:'
    visualize_grid(ex.input_tests[i]['input'])
    print 'Output:'
    visualize_grid(ex.input_tests[i]['output'])

print
print 'Wrong trace:'
visualize_trace(ex.ref_example.input_tests[0]['trace'])

{'dist': 3, 'idx': 5, 'last_event': KarelEvent(timestep=1, type='move', span=(12, 12), cond_span=None, cond_value=None, success=True), 'test_idx': 0, 'example': <datasets.dataset.KarelExample object at 0x7fd62b288c80>, 'trace_length': 4}
DEF run m( IFELSE c( noMarkersPresent c) i( pickMarker i) ELSE e( move e) m)
DEF run m( pickMarker move pickMarker m)
Input:
█████
█155█
█ 11█
█21̅ █
█5  █
█  4█
█████
Output:
█████
█155█
█ ↑1█
█2  █
█5  █
█  4█
█████
Input:
██████████████
█   4 12     █
█ 1          █
█   8     1 3█
█            █
█           5█
█2  1  1  1  █
█1  1 4      █
█    2  81   █
█           9█
█         1  █
█           5█
█  1 3  1 1  █
█       713͔ 9█
█ 11  1  1   █
█11    1 1 8 █
██████████████
Output:
██████████████
█   4 12     █
█ 1          █
█   8     1 3█
█            █
█           5█
█2  1  1  1  █
█1  1 4      █
█    2  81   █
█           9█
█         1  █
█           5█
█  1 3  1 1  █
█       7<2 9█
█ 11  1  1   █
█11    1 1 8 █
██████████████
Input:
███████████

In [30]:
result = the_executor.execute(ex.code_sequence, None, ex.input_tests[0]['input'], record_trace=True, strict=True)
visualize_trace(result.trace)

██████
█1░░ █
█1   █
█4   █
█1͕18 █
█2 1 █
██████
KarelEvent(timestep=1, type='turnLeft', span=(3, 3), cond_span=None, cond_value=None, success=True)
██████
█1░░ █
█1   █
█4   █
█1̅18 █
█2 1 █
██████
KarelEvent(timestep=2, type='repeat', span=(4, 9), cond_span=(5, 5), cond_value=2, success=True)
KarelEvent(timestep=2, type='move', span=(7, 7), cond_span=None, cond_value=None, success=True)
██████
█1░░ █
█1   █
█4̅   █
█118 █
█2 1 █
██████
KarelEvent(timestep=3, type='pickMarker', span=(8, 8), cond_span=None, cond_value=None, success=True)
██████
█1░░ █
█1   █
█3̅   █
█118 █
█2 1 █
██████
KarelEvent(timestep=4, type='repeat', span=(4, 9), cond_span=(5, 5), cond_value=1, success=True)
KarelEvent(timestep=4, type='move', span=(7, 7), cond_span=None, cond_value=None, success=True)
██████
█1░░ █
█1̅   █
█3   █
█118 █
█2 1 █
██████
KarelEvent(timestep=5, type='pickMarker', span=(8, 8), cond_span=None, cond_value=None, success=True)
██████
█1░░ █
█↑   █
█3   █
█118 █
█2 1 █
██████
KarelEvent(

# Old code

In [9]:
def vis(i, j, code=None):
    result = the_executor.execute(
            code if code else ds[i].code_sequence, None, ds[i].input_tests[j]['input'], record_trace=True, strict=True)
    visualize_trace(result.trace)

In [24]:
original_lengths = []
mutated_lengths = []

def trace_length(example):
    trace_lengths = []
    for i in range(5):
        result = the_executor.execute(
            example.code_sequence, None, example.input_tests[i]['input'], record_trace=True, strict=True)
        trace_lengths.append(len(result.trace.grids) + len(result.trace.events))
        
        original_lengths.append((example.idx, i, trace_lengths[-1]))
        
        mutated_test = example.ref_example.input_tests[i]
        mutated_lengths.append((example.idx,
                                example.ref_example.code_sequence,
                                i,
                                len(mutated_test['trace'].grids) + len(mutated_test['trace'].events)))
    #print 'Original:', trace_lengths
    #print 'Mutated: ', [len(it['trace'].grids) + len(it['trace'].events) for it in example.ref_example.input_tests]
    #print

In [25]:
for i in range(10000):
    trace_length(ds[i])
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [39]:
sorted(original_lengths, key=operator.itemgetter(-1), reverse=True)

[(8913, 0, 255),
 (8913, 1, 255),
 (8913, 2, 255),
 (8913, 3, 255),
 (8913, 4, 255),
 (9823, 0, 251),
 (9823, 1, 245),
 (9823, 2, 245),
 (9823, 3, 245),
 (9823, 4, 245),
 (1287, 2, 241),
 (8328, 2, 232),
 (9535, 4, 229),
 (3060, 3, 228),
 (3060, 4, 228),
 (1203, 1, 224),
 (4698, 1, 224),
 (9387, 0, 224),
 (9387, 3, 224),
 (9387, 4, 224),
 (8911, 1, 222),
 (9387, 2, 221),
 (9387, 1, 215),
 (4914, 2, 214),
 (5302, 1, 211),
 (2256, 0, 209),
 (2256, 2, 209),
 (2256, 3, 209),
 (2256, 4, 209),
 (9773, 0, 206),
 (9773, 1, 206),
 (9773, 2, 206),
 (9773, 3, 206),
 (9773, 4, 206),
 (3730, 1, 202),
 (6241, 2, 202),
 (1106, 0, 199),
 (1106, 4, 199),
 (4950, 0, 199),
 (4950, 2, 199),
 (3516, 4, 198),
 (6535, 0, 198),
 (6535, 1, 198),
 (6535, 2, 198),
 (6535, 3, 198),
 (6535, 4, 198),
 (9494, 1, 198),
 (9494, 2, 198),
 (6116, 2, 197),
 (9494, 4, 197),
 (5089, 3, 196),
 (9494, 0, 196),
 (9494, 3, 196),
 (656, 0, 194),
 (656, 4, 194),
 (5399, 0, 194),
 (5399, 1, 194),
 (5399, 2, 194),
 (5399, 3, 194),

In [40]:
' '.join(ds[8913].code_sequence)

u'DEF run m( REPEAT R=4 r( REPEAT R=4 r( REPEAT R=4 r( turnRight r) r) IFELSE c( leftIsClear c) i( putMarker i) ELSE e( turnRight e) r) move REPEAT R=2 r( turnRight r) m)'

In [41]:
vis(8913, 0)

█████████████
█ ░  7      █
█ 1   1     █
█           █
█ 5         █
█    ↑      █
█████████████
KarelEvent(timestep=1, type='repeat', span=(3, 26), cond_span=(4, 4), cond_value=4, success=True)
KarelEvent(timestep=1, type='repeat', span=(6, 14), cond_span=(7, 7), cond_value=4, success=True)
KarelEvent(timestep=1, type='repeat', span=(9, 13), cond_span=(10, 10), cond_value=4, success=True)
KarelEvent(timestep=1, type='turnRight', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████
█ ░  7      █
█ 1   1     █
█           █
█ 5         █
█    >      █
█████████████
KarelEvent(timestep=2, type='repeat', span=(9, 13), cond_span=(10, 10), cond_value=3, success=True)
KarelEvent(timestep=2, type='turnRight', span=(12, 12), cond_span=None, cond_value=None, success=True)
█████████████
█ ░  7      █
█ 1   1     █
█           █
█ 5         █
█    ↓      █
█████████████
KarelEvent(timestep=3, type='repeat', span=(9, 13), cond_span=(10, 10), cond_value=2, success=True)
Kare

In [50]:
for i, code, j, _ in sorted(mutated_lengths, key=operator.itemgetter(-1), reverse=True)[:5]:
    vis(i, j, code)
    print ' '.join(code)
    print
    print '======================='
    print

████████████
█ 1      1░█
█   ░     ░█
█    1 ░1  █
█   ░░░  ░ █
█   1░   1 █
█ 1       ░█
█          █
█   ░░    ░█
█1͔   ░   8 █
████████████
KarelEvent(timestep=1, type='while', span=(3, 25), cond_span=(5, 5), cond_value=True, success=True)
KarelEvent(timestep=1, type='turnRight', span=(8, 8), cond_span=None, cond_value=None, success=True)
████████████
█ 1      1░█
█   ░     ░█
█    1 ░1  █
█   ░░░  ░ █
█   1░   1 █
█ 1       ░█
█          █
█   ░░    ░█
█1̅   ░   8 █
████████████
KarelEvent(timestep=2, type='repeat', span=(9, 24), cond_span=(10, 10), cond_value=5, success=True)
KarelEvent(timestep=2, type='while', span=(12, 21), cond_span=(14, 17), cond_value=False, success=True)
KarelEvent(timestep=2, type='move', span=(22, 22), cond_span=None, cond_value=None, success=True)
████████████
█ 1      1░█
█   ░     ░█
█    1 ░1  █
█   ░░░  ░ █
█   1░   1 █
█ 1       ░█
█          █
█↑  ░░    ░█
█1   ░   8 █
████████████
KarelEvent(timestep=3, type='putMarker', span=(23, 23), cond_span=

█1 41░  41 █
█11444444 ░█
█1         █
█1  ░░    ░█
█1   ░   8 █
████████████
KarelEvent(timestep=213, type='putMarker', span=(23, 23), cond_span=None, cond_value=None, success=True)
████████████
█ 15555551░█
█  5░   5̲ ░█
█  5 1 ░5  █
█115░░░ 4░ █
█1 41░  41 █
█11444444 ░█
█1         █
█1  ░░    ░█
█1   ░   8 █
████████████
KarelEvent(timestep=214, type='repeat', span=(9, 24), cond_span=(10, 10), cond_value=4, success=True)
KarelEvent(timestep=214, type='while', span=(12, 21), cond_span=(14, 17), cond_value=False, success=True)
KarelEvent(timestep=214, type='move', span=(22, 22), cond_span=None, cond_value=None, success=True)
████████████
█ 15555551░█
█  5░   5 ░█
█  5 1 ░5̲  █
█115░░░ 4░ █
█1 41░  41 █
█11444444 ░█
█1         █
█1  ░░    ░█
█1   ░   8 █
████████████
KarelEvent(timestep=215, type='putMarker', span=(23, 23), cond_span=None, cond_value=None, success=True)
████████████
█ 15555551░█
█  5░   5 ░█
█  5 1 ░6̲  █
█115░░░ 4░ █
█1 41░  41 █
█11444444 ░█
█1         █
█1  ░░    ░

█1   ░   8 █
████████████
KarelEvent(timestep=391, type='putMarker', span=(23, 23), cond_span=None, cond_value=None, success=False)
DEF run m( WHILE c( markersPresent c) w( turnRight REPEAT R=5 r( WHILE c( not c( frontIsClear c) c) w( turnLeft w) move putMarker r) w) m)


█████████████
█  5  ░     █
█  1  1     █
█ 4  8     ↓█
█ 1  1  ░  1█
█████████████
KarelEvent(timestep=1, type='turnLeft', span=(3, 3), cond_span=None, cond_value=None, success=True)
█████████████
█  5  ░     █
█  1  1     █
█ 4  8     >█
█ 1  1  ░  1█
█████████████
KarelEvent(timestep=2, type='while', span=(4, 23), cond_span=(6, 6), cond_value=True, success=True)
KarelEvent(timestep=2, type='turnRight', span=(9, 9), cond_span=None, cond_value=None, success=True)
█████████████
█  5  ░     █
█  1  1     █
█ 4  8     ↓█
█ 1  1  ░  1█
█████████████
KarelEvent(timestep=3, type='repeat', span=(10, 21), cond_span=(11, 11), cond_value=7, success=True)
KarelEvent(timestep=3, type='while', span=(13, 20), cond_span=(15, 15), c

█ 1  1  ░ 61█
█████████████
KarelEvent(timestep=161, type='repeat', span=(10, 21), cond_span=(11, 11), cond_value=7, success=True)
KarelEvent(timestep=161, type='while', span=(13, 20), cond_span=(15, 15), cond_value=True, success=True)
KarelEvent(timestep=161, type='move', span=(18, 18), cond_span=None, cond_value=None, success=True)
█████████████
█  5  ░   5͕ █
█  1  1  6 5█
█ 4  8   6 5█
█ 1  1  ░ 61█
█████████████
KarelEvent(timestep=162, type='putMarker', span=(19, 19), cond_span=None, cond_value=None, success=True)
█████████████
█  5  ░   6͕ █
█  1  1  6 5█
█ 4  8   6 5█
█ 1  1  ░ 61█
█████████████
KarelEvent(timestep=163, type='while', span=(13, 20), cond_span=(15, 15), cond_value=True, success=True)
KarelEvent(timestep=163, type='move', span=(18, 18), cond_span=None, cond_value=None, success=True)
█████████████
█  5  ░   6>█
█  1  1  6 5█
█ 4  8   6 5█
█ 1  1  ░ 61█
█████████████
KarelEvent(timestep=164, type='putMarker', span=(19, 19), cond_span=None, cond_value=None, success=T

█  █
█ ░█
█1 █
█ 4̅█
█  █
█  █
█  █
████
KarelEvent(timestep=106, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=2, success=True)
KarelEvent(timestep=106, type='turnLeft', span=(36, 36), cond_span=None, cond_value=None, success=True)
████
█ ░█
█  █
█  █
█ ░█
█1 █
█ 4͔█
█  █
█  █
█  █
████
KarelEvent(timestep=107, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=1, success=True)
KarelEvent(timestep=107, type='turnLeft', span=(36, 36), cond_span=None, cond_value=None, success=True)
████
█ ░█
█  █
█  █
█ ░█
█1 █
█ 4̲█
█  █
█  █
█  █
████
KarelEvent(timestep=108, type='repeat', span=(33, 37), cond_span=(34, 34), cond_value=0, success=True)
KarelEvent(timestep=108, type='putMarker', span=(38, 38), cond_span=None, cond_value=None, success=True)
████
█ ░█
█  █
█  █
█ ░█
█1 █
█ 5̲█
█  █
█  █
█  █
████
KarelEvent(timestep=109, type='if', span=(39, 45), cond_span=(41, 41), cond_value=False, success=True)
KarelEvent(timestep=109, type='repeat', span=(46, 50), cond_span=(

███████
█  1 1█
█    5̲█
█ 1   █
███████
KarelEvent(timestep=108, type='if', span=(39, 45), cond_span=(41, 41), cond_value=False, success=True)
KarelEvent(timestep=108, type='repeat', span=(46, 50), cond_span=(47, 47), cond_value=8, success=True)
KarelEvent(timestep=108, type='turnRight', span=(49, 49), cond_span=None, cond_value=None, success=True)
███████
█  1 1█
█    5͔█
█ 1   █
███████
KarelEvent(timestep=109, type='repeat', span=(46, 50), cond_span=(47, 47), cond_value=7, success=True)
KarelEvent(timestep=109, type='turnRight', span=(49, 49), cond_span=None, cond_value=None, success=True)
███████
█  1 1█
█    5̅█
█ 1   █
███████
KarelEvent(timestep=110, type='repeat', span=(46, 50), cond_span=(47, 47), cond_value=6, success=True)
KarelEvent(timestep=110, type='turnRight', span=(49, 49), cond_span=None, cond_value=None, success=True)
███████
█  1 1█
█    5͕█
█ 1   █
███████
KarelEvent(timestep=111, type='repeat', span=(46, 50), cond_span=(47, 47), cond_value=5, success=True)
KarelE

█             █
█             █
█      ░      █
███████████████
KarelEvent(timestep=108, type='ifElse', span=(15, 28), cond_span=(17, 20), cond_value=False, success=True)
KarelEvent(timestep=108, type='turnLeft', span=(27, 27), cond_span=None, cond_value=None, success=True)
███████████████
█             █
█        4 4  █
█      ░░ ░   █
█      ░ 4 5̅  █
█ ░     1     █
█             █
█             █
█      ░      █
███████████████
KarelEvent(timestep=109, type='while', span=(3, 35), cond_span=(5, 5), cond_value=True, success=True)
KarelEvent(timestep=109, type='move', span=(8, 8), cond_span=None, cond_value=None, success=True)
███████████████
█             █
█        4 4  █
█      ░░ ░↑  █
█      ░ 4 5  █
█ ░     1     █
█             █
█             █
█      ░      █
███████████████
KarelEvent(timestep=110, type='ifElse', span=(9, 34), cond_span=(11, 11), cond_value=True, success=True)
KarelEvent(timestep=110, type='putMarker', span=(14, 14), cond_span=None, cond_value=None, success=